In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter 
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [10]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership", "annual_inc", 
    "verification_status", "pymnt_plan", "dti", "delinq_2yrs", 
    "inq_last_6mths", "open_acc", "pub_rec", "revol_bal", "total_acc", 
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt", 
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", 
    "total_rec_late_fee", "recoveries", "collection_recovery_fee", 
    "last_pymnt_amnt", "collections_12_mths_ex_med", "policy_code", 
    "application_type", "acc_now_delinq", "tot_coll_amt", "tot_cur_bal", 
    "open_acc_6m", "open_act_il", "open_il_12m", "open_il_24m", 
    "mths_since_rcnt_il", "total_bal_il", "il_util", "open_rv_12m", 
    "open_rv_24m", "max_bal_bc", "all_util", "total_rev_hi_lim", "inq_fi", 
    "total_cu_tl", "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", 
    "bc_open_to_buy", "bc_util", "chargeoff_within_12_mths", "delinq_amnt", 
    "mo_sin_old_il_acct", "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", 
    "mo_sin_rcnt_tl", "mort_acc", "mths_since_recent_bc", 
    "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl", 
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0", "num_sats", 
    "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m", 
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", 
    "pub_rec_bankruptcies", "tax_liens", "tot_hi_cred_lim", 
    "total_bal_ex_mort", "total_bc_limit", "total_il_high_credit_limit", 
    "hardship_flag", "debt_settlement_flag",
    "loan_status"
]

target = "loan_status"

In [11]:
# Load the data
validate_df = pd.read_csv(Path('/Users/tjeff/Desktop/CODING/Predicting-Credit-Risk/Resources/Resources/Generator/LoanStats_2019Q1.csv.zip'), skiprows=1)[:-2]
validate_df = validate_df.loc[:, columns].copy()

# Drop the null columns where all values are null
validate_df = validate_df.dropna(axis='columns', how='all')

# Drop the null rows
validate_df = validate_df.dropna()

# Remove the `Issued` loan status
issued_mask = validate_df[target] != 'Issued'
validate_df = validate_df.loc[issued_mask]

# convert interest rate to numerical
validate_df['int_rate'] = validate_df['int_rate'].str.replace('%', '')
validate_df['int_rate'] = validate_df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = dict.fromkeys(['Current', 'Fully Paid'], 'low_risk') 
validate_df = validate_df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period', 'Charged Off'], 'high_risk')    
validate_df = validate_df.replace(x)

low_risk_rows = validate_df[validate_df[target] == 'low_risk']
high_risk_rows = validate_df[validate_df[target] == 'high_risk']

validate_df = pd.concat([low_risk_rows.sample(n=len(high_risk_rows), random_state=37), high_risk_rows])
validate_df = validate_df.reset_index(drop=True)
validate_df = validate_df.rename({target:'target'}, axis="columns")
validate_df

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
0,21500.0,0.2534,635.35,RENT,43000.0,Source Verified,n,25.29,1.0,1.0,...,0.0,0.0,0.0,69356.0,25070.0,5500.0,37956.0,N,N,low_risk
1,20000.0,0.1180,442.88,MORTGAGE,109840.0,Source Verified,n,10.54,0.0,0.0,...,100.0,1.0,0.0,474366.0,41676.0,14000.0,29466.0,N,N,low_risk
2,20000.0,0.0819,628.49,RENT,120000.0,Source Verified,n,24.65,0.0,1.0,...,6.7,0.0,0.0,223583.0,115502.0,183900.0,34583.0,N,N,low_risk
3,35000.0,0.0881,723.32,MORTGAGE,115000.0,Verified,n,18.69,0.0,0.0,...,60.0,0.0,0.0,429362.0,64693.0,24100.0,49562.0,N,N,low_risk
4,6600.0,0.1356,224.17,RENT,80000.0,Source Verified,n,9.66,0.0,0.0,...,0.0,1.0,0.0,39458.0,22521.0,8800.0,24258.0,N,N,low_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,1200.0,0.1614,42.28,MORTGAGE,48000.0,Verified,n,20.53,0.0,0.0,...,0.0,1.0,0.0,104783.0,18449.0,800.0,30883.0,N,N,high_risk
734,10000.0,0.1992,264.50,MORTGAGE,170000.0,Not Verified,n,4.69,1.0,4.0,...,0.0,0.0,0.0,202891.0,37227.0,24300.0,27381.0,N,N,high_risk
735,20000.0,0.0646,612.62,RENT,125000.0,Not Verified,n,7.38,0.0,0.0,...,0.0,0.0,0.0,74500.0,20701.0,44600.0,20000.0,N,N,high_risk
736,7000.0,0.1992,259.86,OWN,20160.0,Not Verified,n,9.86,2.0,1.0,...,100.0,0.0,0.0,17090.0,13007.0,7000.0,4090.0,N,N,high_risk


In [12]:
file_path = Path('/Users/tjeff/Desktop/CODING/Predicting-Credit-Risk/Resources/Resources/Generator/LoanStats_2019Q1.csv.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)


low_risk_rows = df[df[target] == 'low_risk']
high_risk_rows = df[df[target] == 'high_risk']

#df = pd.concat([low_risk_rows, high_risk_rows.sample(n=len(low_risk_rows), replace=True)])
df = pd.concat([low_risk_rows.sample(n=len(high_risk_rows), random_state=42), high_risk_rows])
df = df.reset_index(drop=True)
df = df.rename({target:'target'}, axis="columns")
df

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
0,15000.0,0.0819,471.37,RENT,90000.0,Source Verified,n,21.68,0.0,0.0,...,0.0,0.0,0.0,145765.0,119603.0,18300.0,118365.0,N,N,low_risk
1,12000.0,0.1797,433.65,MORTGAGE,38000.0,Source Verified,n,13.52,0.0,0.0,...,100.0,0.0,0.0,331570.0,22694.0,4500.0,20620.0,N,N,low_risk
2,5000.0,0.1298,168.43,RENT,45000.0,Source Verified,n,10.96,0.0,1.0,...,0.0,0.0,0.0,64853.0,57628.0,3800.0,52753.0,N,N,low_risk
3,24000.0,0.1356,815.15,RENT,50000.0,Source Verified,n,20.67,0.0,1.0,...,0.0,0.0,0.0,89915.0,56317.0,25400.0,47715.0,N,N,low_risk
4,10000.0,0.2340,389.19,RENT,108000.0,Not Verified,n,24.80,0.0,1.0,...,0.0,0.0,0.0,491477.0,99030.0,22000.0,89209.0,N,N,low_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
689,1200.0,0.1614,42.28,MORTGAGE,48000.0,Verified,n,20.53,0.0,0.0,...,0.0,1.0,0.0,104783.0,18449.0,800.0,30883.0,N,N,high_risk
690,10000.0,0.1992,264.50,MORTGAGE,170000.0,Not Verified,n,4.69,1.0,4.0,...,0.0,0.0,0.0,202891.0,37227.0,24300.0,27381.0,N,N,high_risk
691,20000.0,0.0646,612.62,RENT,125000.0,Not Verified,n,7.38,0.0,0.0,...,0.0,0.0,0.0,74500.0,20701.0,44600.0,20000.0,N,N,high_risk
692,7000.0,0.1992,259.86,OWN,20160.0,Not Verified,n,9.86,2.0,1.0,...,100.0,0.0,0.0,17090.0,13007.0,7000.0,4090.0,N,N,high_risk


In [13]:
X=df.drop(columns='target')
X = pd.get_dummies(X)
y=df['target']

In [14]:
X.describe

<bound method NDFrame.describe of      loan_amnt  int_rate  installment  annual_inc    dti  delinq_2yrs  \
0      15000.0    0.0819       471.37     90000.0  21.68          0.0   
1      12000.0    0.1797       433.65     38000.0  13.52          0.0   
2       5000.0    0.1298       168.43     45000.0  10.96          0.0   
3      24000.0    0.1356       815.15     50000.0  20.67          0.0   
4      10000.0    0.2340       389.19    108000.0  24.80          0.0   
..         ...       ...          ...         ...    ...          ...   
689     1200.0    0.1614        42.28     48000.0  20.53          0.0   
690    10000.0    0.1992       264.50    170000.0   4.69          1.0   
691    20000.0    0.0646       612.62    125000.0   7.38          0.0   
692     7000.0    0.1992       259.86     20160.0   9.86          2.0   
693    15000.0    0.2340       583.78     59000.0  30.88          1.0   

     inq_last_6mths  open_acc  pub_rec  revol_bal  ...  \
0               0.0      21.0  

In [15]:
y.value_counts()

high_risk    347
low_risk     347
Name: target, dtype: int64

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [17]:
from imblearn.ensemble import BalancedRandomForestClassifier
random_forest = BalancedRandomForestClassifier(n_estimators = 100)
random_forest = random_forest.fit(X_train, y_train)

In [18]:
from sklearn.metrics import balanced_accuracy_score
y_pred = random_forest.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7202309836718439

In [19]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[47, 34],
       [13, 80]])

In [20]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.78      0.58      0.86      0.67      0.71      0.49        81
   low_risk       0.70      0.86      0.58      0.77      0.71      0.51        93

avg / total       0.74      0.73      0.71      0.72      0.71      0.50       174



In [21]:
feature_names = X.columns
sorted(zip(random_forest.feature_importances_, feature_names), reverse=True)

[(0.07850318410964112, 'total_rec_prncp'),
 (0.06501425823390633, 'last_pymnt_amnt'),
 (0.05592155807329239, 'total_pymnt'),
 (0.055278962500080776, 'total_rec_int'),
 (0.049756859132394346, 'total_pymnt_inv'),
 (0.026134163544275568, 'int_rate'),
 (0.023027241014768874, 'dti'),
 (0.02043818301654599, 'bc_util'),
 (0.0193935306547659, 'annual_inc'),
 (0.019235782328836055, 'max_bal_bc'),
 (0.01825751048703092, 'out_prncp'),
 (0.017936812486312646, 'mo_sin_old_rev_tl_op'),
 (0.0176134353493189, 'mths_since_rcnt_il'),
 (0.017439179412447145, 'installment'),
 (0.01738984065614746, 'out_prncp_inv'),
 (0.01668084008513341, 'tot_hi_cred_lim'),
 (0.016363258186745902, 'revol_bal'),
 (0.01633287916041158, 'il_util'),
 (0.016159691179213554, 'mo_sin_old_il_acct'),
 (0.01598512488771634, 'avg_cur_bal'),
 (0.014219595370192068, 'total_bal_il'),
 (0.013772727936465945, 'total_acc'),
 (0.013353395691982826, 'all_util'),
 (0.013303755218947222, 'tot_cur_bal'),
 (0.012994579969064583, 'mths_since_rec